In [90]:
import pandas as pd
import numpy as np
import taxcalc as tc
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, NumeralTickFormatter
from bokeh.layouts import column
from notebookfunctions import distribution, index_list, percentile
output_notebook()

Loading BokehJS ...

In [91]:
years = range(2018, 2028)
billions = 1e-9
millions = 1e-6

In [92]:
# baseline calculator
calc = tc.Calculator(records=tc.Records('../puf_data/puf.csv'), policy=tc.Policy())

You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [93]:
# new calculator
gfactors = tc.Growfactors('../stage1/growfactors.csv')
wt = pd.read_csv('../puf_stage2/puf_weights.csv.gz')
rec = tc.Records(data='../puf_data/puf.csv',
                 weights=wt,
                 gfactors=gfactors,
                 adjust_ratios='../puf_stage3/puf_ratios.csv')
pol = tc.Policy(gfactors=gfactors)
calc_new = tc.Calculator(records=rec, policy=pol)

You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [94]:
# Pre-TCJA calculator with current weights
pol_2017_cur = tc.Policy()
pretcja = calc.read_json_param_objects('../../Tax-Calculator/taxcalc/reforms/2017_law.json',
                                       None)
pol_2017_cur.implement_reform(pretcja['policy'])
calc_2017_cur = tc.Calculator(records=tc.Records('../puf_data/puf.csv'), policy=pol_2017_cur)

You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [95]:
# Pre-TCJA calculator with new weights
gfactors2 = tc.Growfactors('../stage1/growfactors.csv')
rec_2017 = tc.Records(data='../puf_data/puf.csv',
                      weights=wt,
                      gfactors=gfactors2,
                      adjust_ratios='../puf_stage3/puf_ratios.csv')
pol_2017 = tc.Policy(gfactors=gfactors2)
pol_2017.implement_reform(pretcja['policy'])
calc_2017 = tc.Calculator(records=rec_2017, policy=pol_2017)

You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [96]:
# dictionaries for holding dataframes for each year
cur_dict = {}
new_dict = {}
dict_2017 = {}
dict_2017_cur = {}
var_list = ['c00100', 'combined', 'iitax', 'payrolltax', 's006', 'expanded_income',
            'e00200', 'e00300', 'e00600', 'e00900']
# dictionary for tax liability
liabilities = {
    'combined_cur': [],
    'combined_new': [],
    'combined_pct': [],
    'combined_2017': [],
    'combined_2017_cur': [],
    'income_cur': [],
    'income_new': [],
    'income_pct': [],
    'income_2017': [],
    'income_2017_cur': [],
    'proll_cur': [],
    'proll_new': [],
    'proll_pct': [],
    'proll_2017': [],
    'proll_2017_cur': [],
    'year': []
}

In [97]:
# loop through each year to collect data used later
for year in years:
    calc.advance_to_year(year)
    calc.calc_all()
    calc_new.advance_to_year(year)
    calc_new.calc_all()
    calc_2017.advance_to_year(year)
    calc_2017.calc_all()
    calc_2017_cur.advance_to_year(year)
    calc_2017_cur.calc_all()
    cur_dict[year] = calc.dataframe(var_list)
    new_dict[year] = calc_new.dataframe(var_list)
    dict_2017[year] = calc_2017.dataframe(var_list)
    combined_cur = calc.weighted_total('combined') * billions
    combined_new = calc_new.weighted_total('combined') * billions
    liabilities['combined_cur'].append(combined_cur)
    liabilities['combined_new'].append(combined_new)
    liabilities['combined_pct'].append(combined_new / combined_cur - 1)
    liabilities['combined_2017'].append(calc_2017.weighted_total ('combined') * billions)
    liabilities['combined_2017_cur'].append(calc_2017_cur.weighted_total ('combined') * billions)
    income_cur = calc.weighted_total('iitax') * billions
    income_new = calc_new.weighted_total('iitax') * billions
    liabilities['income_cur'].append(income_cur)
    liabilities['income_new'].append(income_new)
    liabilities['income_pct'].append(income_new / income_cur - 1)
    liabilities['income_2017'].append(calc_2017.weighted_total('iitax') * billions)
    liabilities['income_2017_cur'].append(calc_2017_cur.weighted_total('iitax') * billions)
    proll_cur = calc.weighted_total('payrolltax') * billions
    proll_new = calc_new.weighted_total('payrolltax') * billions
    liabilities['proll_cur'].append(proll_cur)
    liabilities['proll_new'].append(proll_new)
    liabilities['proll_pct'].append(proll_new / proll_cur - 1)
    liabilities['proll_2017'].append(calc_2017.weighted_total('payrolltax') * billions)
    liabilities['proll_2017_cur'].append(calc_2017_cur.weighted_total('payrolltax') * billions)
    liabilities['year'].append(year)

In [98]:
liabilities_cds = ColumnDataSource(liabilities)

In [99]:
liabilities_df = pd.DataFrame(liabilities)
liabilities_df

,combined_2017,combined_2017_cur,combined_cur,combined_new,combined_pct,income_2017,income_2017_cur,income_cur,income_new,income_pct,proll_2017,proll_2017_cur,proll_cur,proll_new,proll_pct,year
0,2875.491027,2851.644957,2689.768405,2715.257453,0.009476,1756.450032,1710.592849,1548.716297,1596.216458,0.030671,1119.040995,1141.052108,1141.052108,1119.040995,-0.019290,2018
1,3067.990355,2959.598647,2797.543098,2903.172462,0.037758,1884.618587,1773.868788,1611.813238,1719.800694,0.066997,1183.371768,1185.729860,1185.729860,1183.371768,-0.001989,2019
2,3200.574742,3067.206098,2903.627397,3031.535782,0.044051,1957.562598,1839.459850,1675.881150,1788.523639,0.067214,1243.012144,1227.746248,1227.746248,1243.012144,0.012434,2020
3,3319.228263,3185.254136,3020.259555,3148.718013,0.042532,2023.287303,1912.830504,1747.835923,1852.777054,0.060041,1295.940960,1272.423632,1272.423632,1295.940960,0.018482,2021
4,3443.152763,3317.982617,3149.509659,3269.981059,0.038251,2095.460423,1996.742573,1828.269615,1922.288719,0.051425,1347.692340,1321.240044,1321.240044,1347.692340,0.020021,2022
5,3580.652281,3462.693095,3291.582994,3405.638452,0.034651,2179.374725,2089.359304,1918.249203,2004.360897,0.044891,1401.277555,1373.333791,1373.333791,1401.277555,0.020347,2023
6,3731.100145,3614.242204,3440.408277,3553.611121,0.032904,2273.666315,2186.365831,2012.531905,2096.177291,0.041562,1457.433830,1427.876372,1427.876372,1457.433830,0.020700,2024
7,3892.122049,3772.202145,3594.662223,3710.937128,0.032347,2375.704740,2287.754058,2110.214136,2194.519818,0.039951,1516.417309,1484.448088,1484.448088,1516.417309,0.021536,2025
8,4058.137575,3938.998364,3966.239039,4086.717688,0.030376,2481.126483,2395.488421,2422.729096,2509.706596,0.035901,1577.011092,1543.509943,1543.509943,1577.011092,0.021705,2026
9,4240.098379,4113.559196,4145.087919,4273.076773,0.030877,2598.900680,2508.437397,2539.966120,2631.879074,0.036187,1641.197699,1605.121799,1605.121799,1641.197699,0.022475,2027


## Current Calculator Comparison

In [100]:
f = figure(title='Tax Liability by Year (Billions)', x_minor_ticks=2)
f.line(x='year', y='combined_cur', color='blue', line_width=2, legend='Combined - Current',
       source=liabilities_cds)
f.line(x='year', y='combined_new', color='cyan', line_width=2, legend='Combined - New',
       source=liabilities_cds)
f.line(x='year', y='income_cur', color='green', line_width=2, legend='Income - Current',
       source=liabilities_cds)
f.line(x='year', y='income_new', color='lime', line_width=2, legend='Income - New',
       source=liabilities_cds)
f.line(x='year', y='proll_cur', color='purple', line_width=2, legend='Payroll - Current',
       source=liabilities_cds)
f.line(x='year', y='proll_new', color='magenta', line_width=2, legend='Payroll - New',
       source=liabilities_cds)
f.circle(x='year', y='combined_cur', color='blue', size=5, legend='Combined - Current',
         source=liabilities_cds)
f.circle(x='year', y='combined_new', color='cyan', size=5, legend='Combined - New',
         source=liabilities_cds)
f.circle(x='year', y='income_cur', color='green', size=5, legend='Income - Current',
         source=liabilities_cds)
f.circle(x='year', y='income_new', color='lime', size=5, legend='Income - New',
         source=liabilities_cds)
f.circle(x='year', y='proll_cur', color='purple', size=5, legend='Payroll - Current',
         source=liabilities_cds)
f.circle(x='year', y='proll_new', color='magenta', size=5, legend='Payroll - New',
         source=liabilities_cds)
f.legend.location = 'top_left'
show(f)

In [102]:
f = figure(title='Percent Change in Tax Liability', x_minor_ticks=2)
f.line(x='year', y='combined_pct', line_width=2, color='blue', legend='Combined',
       source=liabilities_cds)
f.line(x='year', y='income_pct', line_width=2, color='green', legend='Income',
       source=liabilities_cds)
f.line(x='year', y='proll_pct', line_width=2, color='purple', legend='Payroll',
       source=liabilities_cds)
f.yaxis[0].formatter = NumeralTickFormatter(format='0.00%')
show(f)

In [104]:
print('Liabilities under current law')
liabilities_df[['year', 'combined_cur', 'combined_new', 'income_cur', 'income_new',
                'proll_cur', 'proll_new']]

Liabilities under current law


,year,combined_cur,combined_new,income_cur,income_new,proll_cur,proll_new
0,2018,2689.768405,2715.257453,1548.716297,1596.216458,1141.052108,1119.040995
1,2019,2797.543098,2903.172462,1611.813238,1719.800694,1185.729860,1183.371768
2,2020,2903.627397,3031.535782,1675.881150,1788.523639,1227.746248,1243.012144
3,2021,3020.259555,3148.718013,1747.835923,1852.777054,1272.423632,1295.940960
4,2022,3149.509659,3269.981059,1828.269615,1922.288719,1321.240044,1347.692340
5,2023,3291.582994,3405.638452,1918.249203,2004.360897,1373.333791,1401.277555
6,2024,3440.408277,3553.611121,2012.531905,2096.177291,1427.876372,1457.433830
7,2025,3594.662223,3710.937128,2110.214136,2194.519818,1484.448088,1516.417309
8,2026,3966.239039,4086.717688,2422.729096,2509.706596,1543.509943,1577.011092
9,2027,4145.087919,4273.076773,2539.966120,2631.879074,1605.121799,1641.197699


In [105]:
f = figure(title='Tax Liability by Year (Billions) - Pre-TCJA', x_minor_ticks=2)
f.line(x='year', y='combined_2017_cur', color='blue', line_width=2, legend='Combined - Current',
       source=liabilities_cds)
f.line(x='year', y='combined_2017', color='cyan', line_width=2, legend='Combined - New',
       source=liabilities_cds)
f.line(x='year', y='income_2017_cur', color='green', line_width=2, legend='Income - Current',
       source=liabilities_cds)
f.line(x='year', y='income_2017', color='lime', line_width=2, legend='Income - New',
       source=liabilities_cds)
f.line(x='year', y='proll_2017_cur', color='purple', line_width=2, legend='Payroll - Current',
       source=liabilities_cds)
f.line(x='year', y='proll_2017', color='magenta', line_width=2, legend='Payroll - New',
       source=liabilities_cds)
f.circle(x='year', y='combined_2017_cur', color='blue', size=5, legend='Combined - Current',
         source=liabilities_cds)
f.circle(x='year', y='combined_2017', color='cyan', size=5, legend='Combined - New',
         source=liabilities_cds)
f.circle(x='year', y='income_2017_cur', color='green', size=5, legend='Income - Current',
         source=liabilities_cds)
f.circle(x='year', y='income_2017', color='lime', size=5, legend='Income - New',
         source=liabilities_cds)
f.circle(x='year', y='proll_2017_cur', color='purple', size=5, legend='Payroll - Current',
         source=liabilities_cds)
f.circle(x='year', y='proll_2017', color='magenta', size=5, legend='Payroll - New',
         source=liabilities_cds)
f.legend.location = 'top_left'
show(f)

In [107]:
print('Liabilities under 2017 law')
liabilities_df[['year', 'combined_2017_cur', 'combined_2017',
                'income_2017_cur', 'income_2017', 'proll_2017_cur', 'proll_2017']]

Liabilities under 2017 law


,year,combined_2017_cur,combined_2017,income_2017_cur,income_2017,proll_2017_cur,proll_2017
0,2018,2851.644957,2875.491027,1710.592849,1756.450032,1141.052108,1119.040995
1,2019,2959.598647,3067.990355,1773.868788,1884.618587,1185.729860,1183.371768
2,2020,3067.206098,3200.574742,1839.459850,1957.562598,1227.746248,1243.012144
3,2021,3185.254136,3319.228263,1912.830504,2023.287303,1272.423632,1295.940960
4,2022,3317.982617,3443.152763,1996.742573,2095.460423,1321.240044,1347.692340
5,2023,3462.693095,3580.652281,2089.359304,2179.374725,1373.333791,1401.277555
6,2024,3614.242204,3731.100145,2186.365831,2273.666315,1427.876372,1457.433830
7,2025,3772.202145,3892.122049,2287.754058,2375.704740,1484.448088,1516.417309
8,2026,3938.998364,4058.137575,2395.488421,2481.126483,1543.509943,1577.011092
9,2027,4113.559196,4240.098379,2508.437397,2598.900680,1605.121799,1641.197699


### Distributional Analysis

In [108]:
# percentile measures
measure = 'c00100'
perc_cur = {'combined': percentile(cur_dict[2018], 'combined', 100, measure=measure),
            'iitax': percentile(cur_dict[2018], 'iitax', 100, measure=measure),
            'proll': percentile(cur_dict[2018], 'payrolltax', 100, measure=measure),
            'was': percentile(cur_dict[2018], 'e00200', 100, measure=measure),
            'interest': percentile(cur_dict[2018], 'e00300', 100, measure=measure),
            'biz': percentile(cur_dict[2018], 'e00900', 100, measure=measure),
            'index': range(1, 101)}
perc_new = {'combined': percentile(new_dict[2018], 'combined', 100, measure=measure),
            'iitax': percentile(new_dict[2018], 'iitax', 100, measure=measure),
            'proll': percentile(new_dict[2018], 'payrolltax', 100, measure=measure),
            'was': percentile(new_dict[2018], 'e00200', 100, measure=measure),
            'interest': percentile(new_dict[2018], 'e00300', 100, measure=measure),
            'biz': percentile(new_dict[2018], 'e00900', 100, measure=measure),
            'index': range(1, 101)}
# convert to column data source for plotting
perc_cds_cur = ColumnDataSource(perc_cur)
perc_cds_new = ColumnDataSource(perc_new)

In [109]:
f = figure(title='Average Tax Liability by AGI Percentile')
f.line(x='index', y='combined', line_width=2, color='blue',
       legend='Combined - Current', source=perc_cds_cur)
f.line(x='index', y='iitax', line_width=2, color='green',
       legend='Income - Current', source=perc_cds_cur)
f.line(x='index', y='proll', line_width=2, color='purple',
       legend='Payroll - Current', source=perc_cds_cur)
f.line(x='index', y='combined', line_width=2, color='cyan',
       legend='Combined - New', source=perc_cds_new)
f.line(x='index', y='iitax', line_width=2, color='lime',
       legend='Income - New', source=perc_cds_new)
f.line(x='index', y='proll', line_width=2, color='magenta',
       legend='Payroll - New', source=perc_cds_new)
f.legend.location = 'top_left'
f.yaxis[0].formatter = NumeralTickFormatter(format='$0.00a')
show(f)

In [110]:
was = figure(title='Average Wage by AGI Percentile')
was.line(x='index', y='was', line_width=2, color='pink',
       legend='Current', source=perc_cds_cur)
was.line(x='index', y='was', line_width=2, color='lime',
       legend='New', source=perc_cds_new)
was.legend.location = 'top_left'
was.yaxis[0].formatter = NumeralTickFormatter(format='$0.00a')

ints = figure(title='Average Business Income by AGI Percentile')
ints.line(x='index', y='interest', line_width=2, color='pink', legend='Current',
          source=perc_cds_cur)
ints.line(x='index', y='interest', line_width=2, color='lime', legend='New',
          source=perc_cds_new)
ints.legend.location = 'top_left'
ints.yaxis[0].formatter = NumeralTickFormatter(format='$0.00a')

biz = figure(title='Average Business Income by AGI Percentile')
biz.line(x='index', y='biz', line_width=2, color='pink', legend='Current',
         source=perc_cds_cur)
biz.line(x='index', y='biz', line_width=2, color='lime', legend='New',
         source=perc_cds_new)
biz.legend.location = 'top_left'
biz.yaxis[0].formatter = NumeralTickFormatter(format='$0.00a')

show(column([was, ints, biz]))

## Pre-TCJA Comparison

In [111]:
f = figure(title='Tax Liability by Year (Billions)', x_minor_ticks=2)
f.line(x='year', y='combined_2017', color='blue', line_width=2, legend='Combined - Pre-TCJA',
       source=liabilities_cds)
f.line(x='year', y='combined_new', color='cyan', line_width=2, legend='Combined - New',
       source=liabilities_cds)
f.line(x='year', y='income_2017', color='green', line_width=2, legend='Income - Pre-TCJA',
       source=liabilities_cds)
f.line(x='year', y='income_new', color='lime', line_width=2, legend='Income - New',
       source=liabilities_cds)
f.line(x='year', y='proll_2017', color='purple', line_width=2, legend='Payroll - Pre-TCJA',
       source=liabilities_cds)
f.line(x='year', y='proll_new', color='magenta', line_width=2, legend='Payroll - New',
       source=liabilities_cds)
f.circle(x='year', y='combined_2017', color='blue', size=5, legend='Combined - Pre-TCJA',
         source=liabilities_cds)
f.circle(x='year', y='combined_new', color='cyan', size=5, legend='Combined - New',
         source=liabilities_cds)
f.circle(x='year', y='income_2017', color='green', size=5, legend='Income - Pre-TCJA',
         source=liabilities_cds)
f.circle(x='year', y='income_new', color='lime', size=5, legend='Income - New',
         source=liabilities_cds)
f.circle(x='year', y='proll_2017', color='purple', size=5, legend='Payroll - Pre-TCJA',
         source=liabilities_cds)
f.circle(x='year', y='proll_new', color='magenta', size=5, legend='Payroll - New',
         source=liabilities_cds)
f.legend.location = 'top_left'
show(f)